`pidgin.doctesting` provides tooling to apply input transformations to doctest strings.

In [1]:
    import ast, doctest, IPython, textwrap, re, collections, importnb

In [2]:
    class OutputChecker(doctest.OutputChecker):
        def check_output(self, want, got, optionflags):
            return True if want == '...\n' else super().check_output(want, got, optionflags)

In [3]:
    class MultiParser(doctest.DocTestParser):
        def __init__(self, *args, parsers=(doctest.DocTestParser), **kwargs,):
            self._parsers = parsers
            super().__init__(*args, **kwargs)
        
        def get_examples(self, string, name='<string>') -> list:
            return sum([parser.get_examples(string, name) for parser in self._parsers], [])

In [4]:
    class InlineCodeParser(doctest.DocTestParser):
        _EXAMPLE_RE = re.compile(r'(?P<indent>[ *]?)`(?P<source>[\s\S(\\`)]+?)(?P<want>.{0}?)`',
            re.IGNORECASE|re.UNICODE)
        
        def _parse_example(self, m, name, lineno):
            source = m.group('source')
            return source, {doctest.ELLIPSIS: True}, '...', None
        
        def get_examples(self, string, name='<string>'):
            examples = []
            for example in super().get_examples(string, name):
                # This is a tragic way to exclude code fences.
                if example.source.startswith('`'): continue
                if examples  and examples[-1].source.rstrip().endswith('\\'):
                    examples[-1].source += F"""{example.source}"""
                else:
                    
                    examples.append(example)
            return examples
                    

In [5]:
    def run_docstring_examples(
        f, ip, verbose=False, compileflags=None, optionflags=0
    ):
        finder = doctest.DocTestFinder(verbose=verbose, parser=MultiParser(parsers=(
            doctest.DocTestParser(), InlineCodeParser()
        )),recurse=False)
        runner = doctest.DocTestRunner(verbose=verbose, optionflags=optionflags, checker=OutputChecker())
        transformer = importnb.docstrings.TestStrings()
        module = ast.parse(ip.input_transformer_manager.transform_cell(f))
        module.body = [ast.Expr(ast.Str())] + module.body
        transformer.visit(module)
        for node in transformer.strings:
            for test in (doctest_to_ipython(ip, object) for object in finder.find(
                node.s, ip.user_ns['__name__']
            )):
                test.globs = ip.user_ns
                runner.run(test, compileflags=compileflags, clear_globs=False)        
                
        return runner

In [6]:
    def doctest_to_ipython(ip, test):
        for example in test.examples:
            example.source = ip.input_transformer_manager.transform_cell(
                textwrap.indent(example.source, ' '*4)
            )
            if '```' in example.want:
                example.want, ticks, rest = example.want.rpartition('```')
        return test

`run_cell_doctests` runs any `doctest`s found in the raw source.

In [7]:
    def run_cell_doctests(ip, result) -> Exception:
        with IPython.utils.capture.capture_output(
            stdout=True, stderr=False, display=False
        ) as output:
            run_docstring_examples(result.info.raw_cell, ip)

        if output.stdout:
            return DocTestException(output.stdout)

In [8]:
    class DocTestException(BaseException):
        def _render_traceback_(self): return args[0].splitlines()

In [9]:
    def load_ipython_extension(ip):
        try: from . import post_run_cell
        except: import post_run_cell
        ip.input_transformer_manager.cleanup_transforms = [
            object for object in ip.input_transformer_manager.cleanup_transforms
            if object is not IPython.core.inputtransformer2.classic_prompt
        ]
    def unload_ipython_extension(ip):
        try: from . import post_run_cell
        except: import post_run_cell
        post_run_cell.create_load_extension(run_cell_doctests)(ip)
        
    if __name__ == '__main__': 
        %reload_ext pidgin.tangle
        %reload_ext pidgin.display
        unload_ipython_extension(IPython.get_ipython())
        load_ipython_extension(IPython.get_ipython())
